<a href="https://colab.research.google.com/github/PasinduWaidyarathna/Deep-Learning-Mini-Project-03/blob/main/finalTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Necessary Library Imports

In [1]:
import random
import tensorflow as tf
import string
import re
from tensorflow import keras
from tensorflow.keras import layers

Mount the Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install openpyxl

Read the data file

In [3]:
import openpyxl

excel_file = "/content/drive/MyDrive/rowdataentosin.xlsx"
workbook = openpyxl.load_workbook(excel_file)
sheet = workbook.active

for i in range(2, 22):
    cell1 = sheet.cell(row=i, column=1).value
    cell2 = sheet.cell(row=i, column=2).value
    print(f"{cell1}\t{cell2}")

workbook.close()


Go.	යන්න.
Hi.	හායි.
Run!	දුවන්න!
Run.	දුවන්න.
Who?	කවුද?
Fire!	ගිනි!
Help!	උදව්!
Jump!	පනින්න!
Jump.	පනින්න.
Stop!	නවත්වන්න!
Wait!	ඉන්න!
Wait.	ඉන්න.
Go on.	ඉදිරියට යන්න.
Hello!	ආයුබෝවන්!
I ran.	මම දිව්වා.
I try.	මම උත්සාහ කරනවා.
I won!	මම දිනුවා!
Oh no!	ඔහ් නෑ!
Relax.	සන්සුන් වන්න.
Smile.	සිනහව.


Split the English and Sinhala translation pairs

In [5]:
excel_file = "/content/drive/MyDrive/rowdataentosin.xlsx"
workbook = openpyxl.load_workbook(excel_file)
sheet = workbook.active

text_pairs = []
for row in sheet.iter_rows(values_only=True):
    if len(row) >= 2 and row[1] is not None:
        english, sinhala = row[:2]
        sinhala = "[start] " + sinhala + " [end]"
        text_pairs.append((english, sinhala))

for _ in range(3):
    print(random.choice(text_pairs))

workbook.close()


("Get your socks on fast. We're late!", '[start] ඔබේ මේස් වේගයෙන් ලබා ගන්න. අපි පරක්කුයි! [end]')
("It looks like no one's home.", '[start] එය කිසිවෙකු නිවසක නොවන බව පෙනේ. [end]')
('Do you know where Tom is from?', '[start] ටොම් කොහෙන්දැයි ඔබ දන්නවාද? [end]')


Randomize the data

In [6]:
random.shuffle(text_pairs)

Spliting the data into training, validation and Testing

In [7]:
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

print("Total sentences:",len(text_pairs))
print("Training set size:",len(train_pairs))
print("Validation set size:",len(val_pairs))
print("Testing set size:",len(test_pairs))


Total sentences: 102903
Training set size: 72033
Validation set size: 15435
Testing set size: 15435


In [8]:
len(train_pairs)+len(val_pairs)+len(test_pairs)

102903

Removing Punctuations

In [9]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

f"[{re.escape(strip_chars)}]"

'[!"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\\./:;<=>\\?@\\\\\\^_`\\{\\|\\}\\~¿]'

In [10]:
f"{3+5}"

'8'

Vectorizing the English and Sinhala text pairs

In [11]:
def custom_standardization(input_string):
  lowercase = tf.strings.lower(input_string)
  return tf.strings.regex_replace(
      lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 15000
sequence_length = 20

source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_sinhala_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_sinhala_texts)


Preparing datasets for the translation task

In [12]:
batch_size = 64

def format_dataset(eng, sin):
   eng = source_vectorization(eng)
   sin = target_vectorization(sin)
   return ({
         "english": eng,
         "sinhala": sin[:, :-1],
   }, sin[:, 1:])

def make_dataset(pairs):
    eng_texts, sin_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    sin_texts = list(sin_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, sin_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)


for inputs, targets in train_ds.take(1):
   print(f"inputs['english'].shape: {inputs['english'].shape}")
   print(f"inputs['sinhala'].shape: {inputs['sinhala'].shape}")
   print(f"targets.shape: {targets.shape}")




inputs['english'].shape: (64, 20)
inputs['sinhala'].shape: (64, 20)
targets.shape: (64, 20)


In [13]:
print(list(train_ds.as_numpy_iterator())[50])

({'english': array([[   5,    8,  953, ...,    0,    0,    0],
       [   5,   54,    2, ...,    0,    0,    0],
       [4895,   31,  523, ...,    0,    0,    0],
       ...,
       [   2,  321, 2247, ...,    0,    0,    0],
       [   5,   13, 1768, ...,    0,    0,    0],
       [1933,    2, 3209, ...,    0,    0,    0]]), 'sinhala': array([[   2,    5,    1, ...,    0,    0,    0],
       [   2,    5,  295, ...,    0,    0,    0],
       [   2,   34, 6051, ...,    0,    0,    0],
       ...,
       [   2,  938,   91, ...,    0,    0,    0],
       [   2,    5, 2166, ...,    0,    0,    0],
       [   2,  227,  433, ...,    0,    0,    0]])}, array([[   5,    1,    3, ...,    0,    0,    0],
       [   5,  295, 2127, ...,    0,    0,    0],
       [  34, 6051,   20, ...,    0,    0,    0],
       ...,
       [ 938,   91,  470, ...,    0,    0,    0],
       [   5, 2166,  209, ...,    0,    0,    0],
       [ 227,  433, 1447, ...,    0,    0,    0]]))


Transformer encoder implemented as a subclassed Layer

In [14]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
      super().__init__(**kwargs)
      self.embed_dim = embed_dim
      self.dense_dim = dense_dim
      self.num_heads = num_heads
      self.attention = layers.MultiHeadAttention(
           num_heads=num_heads, key_dim=embed_dim)
      self.dense_proj = keras.Sequential(
           [layers.Dense(dense_dim, activation="relu"),
            layers.Dense(embed_dim),]
      )
      self.layernorm_1 = layers.LayerNormalization()
      self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
      if mask is not None:
         mask = mask[:, tf.newaxis, :]
      attention_output = self.attention(
         inputs, inputs, attention_mask=mask)
      proj_input = self.layernorm_1(inputs + attention_output)
      proj_output = self.dense_proj(proj_input)
      return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config


The Transformer decoder

In [15]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
      super().__init__(**kwargs)
      self.embed_dim = embed_dim
      self.dense_dim = dense_dim
      self.num_heads = num_heads
      self.attention_1 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
      self.attention_2 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
      self.dense_proj = keras.Sequential(
          [layers.Dense(dense_dim, activation="relu"),
           layers.Dense(embed_dim),]
      )
      self.layernorm_1 = layers.LayerNormalization()
      self.layernorm_2 = layers.LayerNormalization()
      self.layernorm_3 = layers.LayerNormalization()
      self.supports_masking = True

    def get_config(self):
      config = super().get_config()
      config.update({
          "embed_dim": self.embed_dim,
          "num_heads": self.num_heads,
          "dense_dim": self.dense_dim,
      })
      return config

    def get_causal_attention_mask(self, inputs):
      input_shape = tf.shape(inputs)
      batch_size, sequence_length = input_shape[0], input_shape[1]
      i = tf.range(sequence_length)[:, tf.newaxis]
      j = tf.range(sequence_length)
      mask = tf.cast(i >= j, dtype="int32")
      mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
      mult = tf.concat(
              [tf.expand_dims(batch_size, -1),
               tf.constant([1, 1], dtype=tf.int32)], axis=0)
      return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
      causal_mask = self.get_causal_attention_mask(inputs)
      if mask is not None:
           padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
           padding_mask = tf.minimum(padding_mask, causal_mask)
      else:
           padding_mask = mask
      attention_output_1 = self.attention_1(
           query=inputs,
           value=inputs,
           key=inputs,
           attention_mask=causal_mask)
      attention_output_1 = self.layernorm_1(inputs + attention_output_1)
      attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
       )
      attention_output_2 = self.layernorm_2(
          attention_output_1 + attention_output_2)
      proj_output = self.dense_proj(attention_output_2)
      return self.layernorm_3(attention_output_2 + proj_output)



Positional Encoding

In [16]:
class PositionalEmbedding(layers.Layer):
     def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
         super().__init__(**kwargs)
         self.token_embeddings = layers.Embedding(
             input_dim=input_dim, output_dim=output_dim)
         self.position_embeddings = layers.Embedding(
             input_dim=sequence_length, output_dim=output_dim)
         self.sequence_length = sequence_length
         self.input_dim = input_dim
         self.output_dim = output_dim

     def call(self, inputs):
         length = tf.shape(inputs)[-1]
         positions = tf.range(start=0, limit=length, delta=1)
         embedded_tokens = self.token_embeddings(inputs)
         embedded_positions = self.position_embeddings(positions)
         return embedded_tokens + embedded_positions

     def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

     def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
             "output_dim": self.output_dim,
             "sequence_length": self.sequence_length,
             "input_dim": self.input_dim,
        })
        return config


End-to-end Transformer

In [17]:
embed_dim = 256
dense_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="sinhala")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [18]:
transformer.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 english (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 sinhala (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 positional_embedding (Posi  (None, None, 256)            3845120   ['english[0][0]']             
 tionalEmbedding)                                                                                 
                                                                                                  
 positional_embedding_1 (Po  (None, None, 256)            3845120   ['sinhala[0][0]']         

Training the sequence-to-sequence Transformer

In [19]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
transformer.fit(train_ds, epochs=30, validation_data=val_ds)


Epoch 1/30
1126/1126 [==============================] - 96s 77ms/step - loss: 4.3233 - accuracy: 0.4118 - val_loss: 3.3538 - val_accuracy: 0.5092
Epoch 2/30
1126/1126 [==============================] - 77s 68ms/step - loss: 3.2355 - accuracy: 0.5211 - val_loss: 2.8538 - val_accuracy: 0.5645
Epoch 3/30
1126/1126 [==============================] - 77s 69ms/step - loss: 2.8789 - accuracy: 0.5643 - val_loss: 2.6875 - val_accuracy: 0.5888
Epoch 4/30
1126/1126 [==============================] - 79s 70ms/step - loss: 2.6875 - accuracy: 0.5915 - val_loss: 2.6249 - val_accuracy: 0.6011
Epoch 5/30
1126/1126 [==============================] - 77s 68ms/step - loss: 2.5670 - accuracy: 0.6118 - val_loss: 2.5891 - val_accuracy: 0.6103
Epoch 6/30
1126/1126 [==============================] - 77s 68ms/step - loss: 2.4852 - accuracy: 0.6262 - val_loss: 2.5635 - val_accuracy: 0.6184
Epoch 7/30
1126/1126 [==============================] - 77s 68ms/step - loss: 2.4179 - accuracy: 0.6380 - val_loss: 2.5631 -

Translating new sentences with our Transformer model

In [20]:
sin_vocab = target_vectorization.get_vocabulary()
sin_index_lookup = dict(zip(range(len(sin_vocab)), sin_vocab))
max_decoded_sentence_length = 20


Output Testing and Decoding the output sequence

In [21]:
import numpy as np

def decode_sequence(input_sentence):
     tokenized_input_sentence = source_vectorization([input_sentence])
     decoded_sentence = "[start]"
     for i in range(max_decoded_sentence_length):
       tokenized_target_sentence = target_vectorization(
         [decoded_sentence])[:, :-1]
       predictions = transformer(
         [tokenized_input_sentence, tokenized_target_sentence])
       sampled_token_index = np.argmax(predictions[0, i, :])
       sampled_token = sin_index_lookup[sampled_token_index]
       decoded_sentence += " " + sampled_token
       if sampled_token == "[end]": break
     return decoded_sentence

Transformer translating output

In [28]:
test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(5):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))



-
You must read this book.
[start] ඔබ මෙම පොත කියවිය යුතුයි [end]
-
How's business?
[start] සමාගම කෙසේද [end]
-
It isn't new.
[start] එය අලුත් නොවේ [end]
-
He inserted the key in the lock.
[start] ඔහු යතුර [UNK] දොර වසා ගත්තේය [end]
-
We don't want to be separated.
[start] අපට වෙන් වීමට අවශ්ය නැත [end]


Evaluation using the BLEU score

In [26]:
from nltk.translate.bleu_score import sentence_bleu

test_eng_texts = [pair[0] for pair in test_pairs]
test_sin_texts = [pair[1] for pair in test_pairs]
score = 0
bleu  = 0
for i in range(20):
    candidate = decode_sequence(test_eng_texts[i])
    reference = test_sin_texts[i].lower()
    print(candidate,reference)
    score = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0))
    bleu+=score
    print(f"Score:{score}")
print(f"\nBLEU score : {round(bleu,2)}/20")

[start] නායකයා නායකයා අනෙක් පැත්තේ ය [end] [start] පල්ලිය වීදියේ අනෙක් පැත්තේ ය. [end]
Score:0.47619047619047616


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

[start] කරුණාකර අපට කෝපි දෙකක් ගෙන ඒමට පොරොන්දු වන්න [end] [start] කරුණාකර අපට කෝපි කෝප්ප දෙකක් ගෙනෙන්න. [end]
Score:0.43103448275862066
[start] මම ඇතුලට ටිකක් එන්නම් [end] [start] මම මොහොතකට සූදානම්ව සිටිමි. [end]
Score:0.4857142857142857
[start] ඔබ කරුණාකර දොර වසා සිටිනවාද [end] [start] කරුණාකර ඔබ දොර වසා දමනවාද? [end]
Score:0.5365853658536586
[start] මම හිතන්නේ ඔබට හැකි ඉක්මනින් ටොම් අල්ලා ගැනීමට අවශ්යයි [end] [start] මම හිතන්නේ ඔබට හැකි ඉක්මනින් ටොම් සම්බන්ධ කර ගැනීමට අවශ්යයි. [end]
Score:0.4411764705882353
[start] මට මේ සඳහා මගේ [UNK] ආවා [end] [start] මට මේ අධ්යයන වාරය සඳහා මගේ පෙළපොත් ලැබුණා. [end]
Score:0.5
[start] ටොම්ට වෙන වෙන්නේ නැහැ [end] [start] ටොම්ට වෙනත් තැනක් නැත. [end]
Score:0.5142857142857142
[start] ඔහු වියදම් [UNK] සමඟ ඉදිරියට සිටියේය [end] [start] රාජකාරිය පැහැර හැරීම සඳහා ඔහු උසාවියේදී විවාහ විය. [end]
Score:0.42
[start] ඔහු පැයක් බලා සිටියේය [end] [start] ඔහු පැයක් බලා සිටී. [end]
Score:0.6857142857142857
[start] ටොම් පැවසුවේ මරියාට කිසිවක් කිව නොහැකි බවයි මට ඒක

Calculate accuracy for 20 new sentences

In [29]:
manualTest = [
    ("I love learning new languages.", "මම අලුත් භාෂා ඉගෙන ගන්න ආසයි."),
    ("This is a beautiful day.", "මේක ලස්සන දවසක්."),
    ("She sings very well.", "ඇය ඉතා හොඳින් ගායනා කරයි."),
    ("They are going to the park.", "ඔවුන් උද්යානයට යනවා."),
    ("The book is on the table.", "පොත මේසය උඩ."),
    ("We need to finish this project.", "අපි මේ ව්‍යාපෘතිය අවසන් කළ යුතුයි."),
    ("He is a good friend.", "ඔහු හොඳ මිතුරෙකි."),
    ("I want to travel around the world.", "මට ලොව වටා සංචාරය කිරීමට අවශ්‍යයි."),
    ("The cat is sleeping on the sofa.", "බළලා සෝෆා මත නිදාගෙන සිටී."),
    ("She cooks delicious food.", "ඇය රසවත් කෑම උයනවා."),
    ("We have a lot of work to do.", "අපිට කරන්න වැඩ ගොඩක් තියෙනවා."),
    ("He speaks Spanish fluently.", "ඔහු චතුර ලෙස ස්පාඤ්ඤ භාෂාව කතා කරයි."),
    ("The concert starts at 7 PM.", "ප්‍රසංගය සවස 7 ට ආරම්භ වේ."),
    ("I need to buy some groceries.", "මට සිල්ලර බඩු ටිකක් ගන්න ඕන."),
    ("She is studying for her exam.", "ඇය විභාගයට පාඩම් කරයි."),
    ("They are watching a movie.", "එයාලා ෆිල්ම් එකක් බලනවා."),
    ("He plays the guitar very well.", "ඔහු ඉතා හොඳින් ගිටාර් වාදනය කරයි."),
    ("We are going to the beach tomorrow.", "අපි හෙට මුහුදු වෙරළට යනවා."),
    ("The museum is closed on Mondays.", "සඳුදා දිනවල කෞතුකාගාරය වසා ඇත."),
    ("I am going to visit my family.", "මම මගේ පවුල බලන්න යනවා.")
]

from nltk.translate.bleu_score import sentence_bleu

testENTexts = [pair[0] for pair in manualTest]
testSITexts = [pair[1] for pair in manualTest]
score = 0
bleu  = 0
for i in range(20):
    candidate = decode_sequence(testENTexts[i])
    reference = testSITexts[i].lower()
    print(candidate,reference)
    score = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0))
    bleu+=score
    print(f"Score:{score}")
print(f"\nBLEU score : {round(bleu,2)}/20")




[start] මම නව භාෂා ඉගෙන ගැනීමට කැමතියි [end] මම අලුත් භාෂා ඉගෙන ගන්න ආසයි.
Score:0.2727272727272727
[start] මෙය ලස්සන දවසක් [end] මේක ලස්සන දවසක්.
Score:0.3103448275862069
[start] ඇය ඉතා හොඳින් ගායනා කළහ [end] ඇය ඉතා හොඳින් ගායනා කරයි.
Score:0.3783783783783784
[start] ඔවුන් උද්යානයේ යනවා [end] ඔවුන් උද්යානයට යනවා.
Score:0.30303030303030304
[start] පොත මේසය මත ඇත [end] පොත මේසය උඩ.
Score:0.2857142857142857
[start] අපට මෙම ගැටලුව අවසන් කළ යුතුය [end] අපි මේ ව්‍යාපෘතිය අවසන් කළ යුතුයි.
Score:0.3023255813953489
[start] ඔහු හොඳ මිතුරෙකු [end] ඔහු හොඳ මිතුරෙකි.
Score:0.4
[start] මට ලොව පුරා බොස්ටන් වෙත අවශ්යයි [end] මට ලොව වටා සංචාරය කිරීමට අවශ්‍යයි.
Score:0.31111111111111106
[start] බළලා බර මත නිදාගෙන සිටී [end] බළලා සෝෆා මත නිදාගෙන සිටී.
Score:0.40540540540540543
[start] ඇය රසවත් ආහාර [UNK] [end] ඇය රසවත් කෑම උයනවා.
Score:0.2727272727272727
[start] අපිට කරන්න වැඩ තියෙනවා [end] අපිට කරන්න වැඩ ගොඩක් තියෙනවා.
Score:0.4444444444444444
[start] ඔහු ස්පා spanish ් [UNK] භාෂාව කථා කරයි [end] ඔහු ච